In [68]:
import pandas as pd
from torch.utils.data import Dataset,DataLoader

# read reviews.parquet file
reviews_df = pd.read_parquet('data/reviews.parquet')

# read recipes.parquet file
recipes_df = pd.read_parquet('data/recipes.parquet')

In [69]:
# print the first 5 rows of reviews_df
reviews_df.head()


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25 21:44:00,2000-01-25 21:44:00
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17 16:49:59,2001-10-17 16:49:59
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25 09:00:00,2000-02-25 09:00:00
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13 21:15:00,2000-03-13 21:15:00
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28 12:51:00,2000-03-28 12:51:00


In [70]:
recipes_df.head()


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38.0,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39.0,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40.0,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41.0,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42.0,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."


In [71]:
data = pd.merge(reviews_df, recipes_df, on='RecipeId')


# print the merged dataframe
data.head()

,ReviewId,RecipeId,AuthorId_x,AuthorName_x,Rating,Review,DateSubmitted,DateModified,Name,AuthorId_y,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25 21:44:00,2000-01-25 21:44:00,Jalapeno Pepper Poppers,1545,...,4.9,23.7,172.5,3.2,0.6,0.9,4.3,24.0,None,"[In a mixing bowl, combine cheeses, bacon and ..."
1,250,992,4242,marie m,5,my family loved these. fresher than any bough...,2001-01-02 16:19:29,2001-01-02 16:19:29,Jalapeno Pepper Poppers,1545,...,4.9,23.7,172.5,3.2,0.6,0.9,4.3,24.0,None,"[In a mixing bowl, combine cheeses, bacon and ..."
2,242612,992,205100,mommyoftwo,5,Wow! Awesome - Awesome - Awesome! And I don'...,2006-01-27 06:09:46,2006-01-27 06:09:46,Jalapeno Pepper Poppers,1545,...,4.9,23.7,172.5,3.2,0.6,0.9,4.3,24.0,None,"[In a mixing bowl, combine cheeses, bacon and ..."
3,463486,992,559815,karen.nash-horton,4,This is a good recipe. I much prefer the oven ...,2007-08-15 11:04:13,2007-08-15 11:04:13,Jalapeno Pepper Poppers,1545,...,4.9,23.7,172.5,3.2,0.6,0.9,4.3,24.0,None,"[In a mixing bowl, combine cheeses, bacon and ..."
4,643932,992,743849,NELady,5,"GREAT recipe! I used fat free cream cheese, p...",2008-06-02 18:24:08,2008-06-02 18:24:08,Jalapeno Pepper Poppers,1545,...,4.9,23.7,172.5,3.2,0.6,0.9,4.3,24.0,None,"[In a mixing bowl, combine cheeses, bacon and ..."


In [72]:
df = data[["RecipeId", "AuthorId_y", "Rating", "Calories", "ReviewCount"]]

# remove duplicates without considering the first column
df = df[~df.iloc[:, 1:].duplicated(keep='first', subset=df.columns[1:])]

# reset the index and drop the original index column
df = df.reset_index(drop=True)

# print the updated dataframe
df.head()

,RecipeId,AuthorId_y,Rating,Calories,ReviewCount
0,992,1545,5,111.4,15.0
1,992,1545,4,111.4,15.0
2,4384,1920,4,4.6,3.0
3,4384,1920,5,4.6,3.0
4,4523,1932,2,420.7,10.0


In [73]:


class RecipeDataset(Dataset):
    def __init__(self, data):
        self.recipe_ids = data["RecipeId"].values
        self.author_ids = data["AuthorId_y"].values
        self.ratings = data["Rating"].values
        self.calories = data["Calories"].values
        self.review_counts = data["ReviewCount"].values

    def __len__(self):
        return len(self.recipe_ids)

    def __getitem__(self, idx):
        recipe_id = self.recipe_ids[idx]
        author_id = self.author_ids[idx]
        rating = self.ratings[idx]
        calories = self.calories[idx]
        review_count = self.review_counts[idx]
        return (recipe_id, author_id, rating, calories, review_count), rating


In [74]:

batch_size = 32
train_data = data.sample(frac=0.8, random_state=42)
valid_data = data.drop(train_data.index)
train_dataset = RecipeDataset(train_data)
valid_dataset = RecipeDataset(valid_data)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


In [75]:
import torch.nn as nn
import torch.optim as optim
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class RecipeModel(torch.nn.Module):
    def __init__(self, num_recipes, num_authors):
        super().__init__()
        self.recipe_embedding = torch.nn.Embedding(num_recipes, 16)
        self.author_embedding = torch.nn.Embedding(num_authors, 16)
        self.fc1 = torch.nn.Linear(80, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 1)
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.2)

    def forward(self, recipe_id, author_id, rating, calories, review_counts):
        # recipe_embedding_size = self.recipe_embedding.weight.size(0)  # get the size of the embedding tensor
        # valid_indices = recipe_id < recipe_embedding_size  # get a boolean mask of valid indices
        # recipe_id_filtered = recipe_id[valid_indices]  # filter the recipe_id tensor
        # recipe_embedded = self.recipe_embedding(recipe_id_filtered)
        #
        # author_embedding_size = self.author_embedding.weight.size(0)  # get the size of the embedding tensor
        # valid_indices = author_id < author_embedding_size  # get a boolean mask of valid indices
        # author_id_filtered = author_id[valid_indices]  # filter the author_id tensor
        # author_embedded = self.author_embedding(author_id_filtered)
        recipe_embedded = self.recipe_embedding(recipe_id)
        author_embedded = self.author_embedding(author_id)
        x = torch.cat([recipe_embedded, author_embedded, rating.unsqueeze(-1), calories.unsqueeze(-1), review_counts.unsqueeze(-1)], dim=-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x.view(-1)

model = RecipeModel(num_recipes=len(data["RecipeId"].unique())+1, num_authors=len(data["AuthorId_y"].unique())+1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch, targets in train_loader:
        optimizer.zero_grad()
        batch = [b.to(device) for b in batch]
        targets = targets.to(device)
        preds = model(*batch)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch[0].shape[0]
    train_loss /= len(train_dataset)

    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for batch, targets in valid_loader:
            batch = [b.to(device) for b in batch]
            targets = targets.to(device)
            preds = model(*batch)
            loss = criterion(preds, targets)
            valid_loss += loss.item() * batch[0].shape[0]
        valid_loss /= len(valid_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")



IndexError: index out of range in self

In [ ]:
author_id = 250
recipe_ids = data["recipeId"].unique()

# Get recipes that the user has already rated
user_rated_recipe_ids = data[data["AuthorId_y"] == author_id]["RecipeId"].unique()

# Predict ratings for all recipes that the user hasn't rated
ratings = []
for recipe_id in recipe_ids:
    if recipe_id not in user_rated_recipe_ids:
        input = (torch.tensor(recipe_id), torch.tensor(author_id), torch.tensor(0), torch.tensor(0), torch.tensor(0))
        rating = model(*input.to(device))
        ratings.append(rating.item())

# Select top-rated recipes
top_recipe_ids = [recipe_ids[i] for i in sorted(range(len(ratings)), key=lambda i: ratings[i], reverse=True)[:10]]
